In [1]:
import numpy as np

class FuzzySet:
    """Class to represent a fuzzy set with membership functions"""

    def __init__(self, elements, memberships):
        """
        Initialize a fuzzy set
        elements: list of elements in the universe
        memberships: list of membership values (0 to 1)
        """
        if len(elements) != len(memberships):
            raise ValueError("Elements and memberships must have the same length")

        self.elements = elements
        self.memberships = np.array(memberships)

    def __repr__(self):
        pairs = [f"({e}, {m:.1f})" for e, m in zip(self.elements, self.memberships)]
        return "{ " + ", ".join(pairs) + " }"

    def union(self, other):
        """Union of two fuzzy sets: max(μA(x), μB(x))"""
        if self.elements != other.elements:
            raise ValueError("Fuzzy sets must be defined on the same universe")

        new_memberships = np.maximum(self.memberships, other.memberships)
        return FuzzySet(self.elements, new_memberships)

    def intersection(self, other):
        """Intersection of two fuzzy sets: min(μA(x), μB(x))"""
        if self.elements != other.elements:
            raise ValueError("Fuzzy sets must be defined on the same universe")

        new_memberships = np.minimum(self.memberships, other.memberships)
        return FuzzySet(self.elements, new_memberships)

    def complement(self):
        """Complement of a fuzzy set: 1 - μA(x)"""
        new_memberships = 1 - self.memberships
        return FuzzySet(self.elements, new_memberships)


class FuzzyRelation:
    """Class to represent fuzzy relations"""

    def __init__(self, matrix, x_elements, y_elements):
        """
        Initialize a fuzzy relation
        matrix: numpy array representing the relation
        x_elements: elements of set X
        y_elements: elements of set Y
        """
        self.matrix = np.array(matrix)
        self.x_elements = x_elements
        self.y_elements = y_elements

    def __repr__(self):
        return f"Fuzzy Relation Matrix:\n{self.matrix}"

    @staticmethod
    def cartesian_product(fuzzy_set_a, fuzzy_set_b):
        """
        Create fuzzy relation from Cartesian product of two fuzzy sets
        R(x,y) = min(μA(x), μB(y))
        """
        n = len(fuzzy_set_a.elements)
        m = len(fuzzy_set_b.elements)

        relation_matrix = np.zeros((n, m))

        for i in range(n):
            for j in range(m):
                relation_matrix[i][j] = min(fuzzy_set_a.memberships[i],
                                           fuzzy_set_b.memberships[j])

        return FuzzyRelation(relation_matrix,
                           fuzzy_set_a.elements,
                           fuzzy_set_b.elements)

    @staticmethod
    def max_min_composition(r1, r2):
        """
        Max-min composition of two fuzzy relations
        R(x,z) = max_y[min(R1(x,y), R2(y,z))]
        R1: relation between X and Y
        R2: relation between Y and Z
        """
        if r1.y_elements != r2.x_elements:
            raise ValueError("Relations cannot be composed: Y dimensions don't match")

        x_size = len(r1.x_elements)
        z_size = len(r2.y_elements)
        y_size = len(r1.y_elements)

        result_matrix = np.zeros((x_size, z_size))

        for i in range(x_size):
            for j in range(z_size):
                # For each (x,z) pair, find max over all y of min(R1(x,y), R2(y,z))
                max_val = 0
                for k in range(y_size):
                    min_val = min(r1.matrix[i][k], r2.matrix[k][j])
                    max_val = max(max_val, min_val)
                result_matrix[i][j] = max_val

        return FuzzyRelation(result_matrix, r1.x_elements, r2.y_elements)


def print_section(title):
    """Helper function to print section headers"""
    print("\n" + "="*60)
    print(title)
    print("="*60)


def main():
    """Main function demonstrating fuzzy set operations and relations"""

    print_section("FUZZY SET OPERATIONS")

    # Define fuzzy sets A and B
    A = FuzzySet([1, 2, 3], [0.2, 0.7, 1.0])
    B = FuzzySet([1, 2, 3], [0.5, 0.4, 0.9])

    print(f"\nFuzzy Set A: {A}")
    print(f"Fuzzy Set B: {B}")

    # Union
    union_AB = A.union(B)
    print(f"\nUnion (A ∪ B): {union_AB}")

    # Intersection
    intersection_AB = A.intersection(B)
    print(f"Intersection (A ∩ B): {intersection_AB}")

    # Complement
    complement_A = A.complement()
    print(f"Complement of A (A'): {complement_A}")

    print_section("FUZZY RELATIONS - CARTESIAN PRODUCT")

    # Create fuzzy relation using Cartesian product
    R = FuzzyRelation.cartesian_product(A, B)
    print(f"\nFuzzy Relation R (A × B):")
    print(R.matrix)
    print(f"\nRows represent elements of A: {A.elements}")
    print(f"Columns represent elements of B: {B.elements}")

    print_section("MAX-MIN COMPOSITION")

    # Example: Create two relations for composition
    # R1: relation between X={1,2,3} and Y={a,b,c}
    R1_matrix = [
        [0.3, 0.8, 0.5],
        [0.6, 0.2, 0.7],
        [0.4, 0.9, 0.3]
    ]
    R1 = FuzzyRelation(R1_matrix, [1, 2, 3], ['a', 'b', 'c'])

    # R2: relation between Y={a,b,c} and Z={x,y}
    R2_matrix = [
        [0.7, 0.4],
        [0.5, 0.8],
        [0.6, 0.3]
    ]
    R2 = FuzzyRelation(R2_matrix, ['a', 'b', 'c'], ['x', 'y'])

    print("\nRelation R1 (X × Y):")
    print(R1.matrix)
    print(f"X = {R1.x_elements}, Y = {R1.y_elements}")

    print("\nRelation R2 (Y × Z):")
    print(R2.matrix)
    print(f"Y = {R2.x_elements}, Z = {R2.y_elements}")

    # Compute max-min composition
    R_composed = FuzzyRelation.max_min_composition(R1, R2)
    print("\nMax-Min Composition R = R1 ∘ R2 (X × Z):")
    print(R_composed.matrix)
    print(f"X = {R_composed.x_elements}, Z = {R_composed.y_elements}")

    # Show detailed calculation for one element
    print("\n" + "-"*60)
    print("Example calculation for R(1, x):")
    print("R(1, x) = max{ min(R1(1,a), R2(a,x)), min(R1(1,b), R2(b,x)), min(R1(1,c), R2(c,x)) }")
    print(f"        = max{{ min(0.3, 0.7), min(0.8, 0.5), min(0.5, 0.6) }}")
    print(f"        = max{{ 0.3, 0.5, 0.5 }}")
    print(f"        = 0.5")


if __name__ == "__main__":
    main()

# ============================================================================
# FUZZY SETS AND FUZZY RELATIONS WITH MAX-MIN COMPOSITION
# ============================================================================
#
# GOAL: Implement fuzzy set operations and fuzzy relations for handling
#       uncertainty and vagueness in data
#
# KEY CONCEPTS:
#
# FUZZY SETS:
# - Unlike crisp sets (element is IN or OUT), fuzzy sets allow partial membership
# - Membership values range from 0 (not a member) to 1 (full member)
# - Example: Temperature 22°C might be 0.7 "warm" and 0.3 "hot"
#
# FUZZY OPERATIONS:
# 1. Union (OR): Takes maximum membership - "either A or B"
# 2. Intersection (AND): Takes minimum membership - "both A and B"
# 3. Complement (NOT): 1 minus membership - "not A"
#
# FUZZY RELATIONS:
# - Represent relationships between elements of two fuzzy sets
# - Stored as a matrix where each cell shows degree of relationship
#
# MAX-MIN COMPOSITION:
# - Combines two relations to find indirect relationships
# - Like chaining: "X relates to Y" and "Y relates to Z" → "X relates to Z"
#
# ============================================================================
# IMPORT
# ============================================================================
#
# import numpy as np
# - Numerical operations and array handling
# - Used for element-wise operations on membership values
#
# ============================================================================
# FUZZY SET CLASS
# ============================================================================
#
# class FuzzySet:
#     """Class to represent a fuzzy set with membership functions"""
#
# - Encapsulates elements and their membership degrees
# - Provides operations: union, intersection, complement
#
#     def __init__(self, elements, memberships):
#
# - Constructor to create a fuzzy set
# - elements: List of universe elements (e.g., [1, 2, 3])
# - memberships: Corresponding membership values (e.g., [0.2, 0.7, 1.0])
#
#         if len(elements) != len(memberships):
#             raise ValueError("Elements and memberships must have the same length")
#
# - Validation: Ensure each element has a membership value
# - Prevents inconsistent fuzzy sets
#
#         self.elements = elements
#         self.memberships = np.array(memberships)
#
# - Store elements as list
# - Convert memberships to numpy array for efficient operations
#
# EXAMPLE FUZZY SET:
# Elements: [1, 2, 3]
# Memberships: [0.2, 0.7, 1.0]
# Interpretation:
#   - Element 1 belongs with degree 0.2 (20% membership)
#   - Element 2 belongs with degree 0.7 (70% membership)
#   - Element 3 belongs with degree 1.0 (100% membership - full member)
#
#     def __repr__(self):
#         pairs = [f"({e}, {m:.1f})" for e, m in zip(self.elements, self.memberships)]
#         return "{ " + ", ".join(pairs) + " }"
#
# - String representation for printing
# - zip() pairs each element with its membership
# - List comprehension creates formatted pairs
# - .1f formats floats to 1 decimal place
#
# OUTPUT EXAMPLE: "{ (1, 0.2), (2, 0.7), (3, 1.0) }"
#
# ============================================================================
# FUZZY UNION OPERATION
# ============================================================================
#
#     def union(self, other):
#         """Union of two fuzzy sets: max(μA(x), μB(x))"""
#
# - Represents logical OR: "element belongs to A OR B"
# - Takes maximum membership from both sets
# - Symbol: A ∪ B
#
#         if self.elements != other.elements:
#             raise ValueError("Fuzzy sets must be defined on the same universe")
#
# - Both sets must have same elements (same universe)
# - Can't union sets with different element spaces
#
#         new_memberships = np.maximum(self.memberships, other.memberships)
#
# - np.maximum: Element-wise maximum
# - For each element, takes higher membership value
#
# EXAMPLE:
# A = {(1, 0.2), (2, 0.7), (3, 1.0)}
# B = {(1, 0.5), (2, 0.4), (3, 0.9)}
#
# Union A ∪ B:
# Element 1: max(0.2, 0.5) = 0.5
# Element 2: max(0.7, 0.4) = 0.7
# Element 3: max(1.0, 0.9) = 1.0
#
# Result: {(1, 0.5), (2, 0.7), (3, 1.0)}
#
# INTERPRETATION: Take the stronger membership from either set
#
#         return FuzzySet(self.elements, new_memberships)
#
# - Creates new fuzzy set with union memberships
#
# ============================================================================
# FUZZY INTERSECTION OPERATION
# ============================================================================
#
#     def intersection(self, other):
#         """Intersection of two fuzzy sets: min(μA(x), μB(x))"""
#
# - Represents logical AND: "element belongs to A AND B"
# - Takes minimum membership from both sets
# - Symbol: A ∩ B
#
#         if self.elements != other.elements:
#             raise ValueError("Fuzzy sets must be defined on the same universe")
#
#         new_memberships = np.minimum(self.memberships, other.memberships)
#
# - np.minimum: Element-wise minimum
# - For each element, takes lower membership value
#
# EXAMPLE:
# A = {(1, 0.2), (2, 0.7), (3, 1.0)}
# B = {(1, 0.5), (2, 0.4), (3, 0.9)}
#
# Intersection A ∩ B:
# Element 1: min(0.2, 0.5) = 0.2
# Element 2: min(0.7, 0.4) = 0.4
# Element 3: min(1.0, 0.9) = 0.9
#
# Result: {(1, 0.2), (2, 0.4), (3, 0.9)}
#
# INTERPRETATION: Take the weaker membership (both must agree)
#
#         return FuzzySet(self.elements, new_memberships)
#
# ============================================================================
# FUZZY COMPLEMENT OPERATION
# ============================================================================
#
#     def complement(self):
#         """Complement of a fuzzy set: 1 - μA(x)"""
#
# - Represents logical NOT: "element does NOT belong to A"
# - Subtracts membership from 1
# - Symbol: A' or Ā
#
#         new_memberships = 1 - self.memberships
#
# - Inverts membership values
# - High membership becomes low, and vice versa
#
# EXAMPLE:
# A = {(1, 0.2), (2, 0.7), (3, 1.0)}
#
# Complement A':
# Element 1: 1 - 0.2 = 0.8
# Element 2: 1 - 0.7 = 0.3
# Element 3: 1 - 1.0 = 0.0
#
# Result: {(1, 0.8), (2, 0.3), (3, 0.0)}
#
# INTERPRETATION: If element is 70% in A, it's 30% NOT in A
#
#         return FuzzySet(self.elements, new_memberships)
#
# ============================================================================
# FUZZY RELATION CLASS
# ============================================================================
#
# class FuzzyRelation:
#     """Class to represent fuzzy relations"""
#
# - Represents relationships between elements of two sets
# - Stored as a matrix: rows=first set, columns=second set
# - Each cell contains degree of relationship (0 to 1)
#
#     def __init__(self, matrix, x_elements, y_elements):
#
# - matrix: 2D array of relationship strengths
# - x_elements: Elements of first set (rows)
# - y_elements: Elements of second set (columns)
#
#         self.matrix = np.array(matrix)
#         self.x_elements = x_elements
#         self.y_elements = y_elements
#
# EXAMPLE RELATION:
#        Y: [a,   b,   c]
# X: 1  [0.3, 0.8, 0.5]
#    2  [0.6, 0.2, 0.7]
#    3  [0.4, 0.9, 0.3]
#
# Interpretation:
# - R(1,a) = 0.3: Element 1 relates to 'a' with degree 0.3
# - R(2,c) = 0.7: Element 2 relates to 'c' with degree 0.7
#
#     def __repr__(self):
#         return f"Fuzzy Relation Matrix:\n{self.matrix}"
#
# - String representation showing the matrix
#
# ============================================================================
# CARTESIAN PRODUCT
# ============================================================================
#
#     @staticmethod
#     def cartesian_product(fuzzy_set_a, fuzzy_set_b):
#         """
#         Create fuzzy relation from Cartesian product of two fuzzy sets
#         R(x,y) = min(μA(x), μB(y))
#         """
#
# - Creates relation by pairing every element of A with every element of B
# - Relationship strength = minimum of both memberships
# - Formula: R(x,y) = min(μA(x), μB(y))
#
#         n = len(fuzzy_set_a.elements)
#         m = len(fuzzy_set_b.elements)
#
# - n: Number of elements in set A
# - m: Number of elements in set B
# - Result will be n×m matrix
#
#         relation_matrix = np.zeros((n, m))
#
# - Initialize empty relation matrix
# - All relationships start at 0
#
#         for i in range(n):
#             for j in range(m):
#                 relation_matrix[i][j] = min(fuzzy_set_a.memberships[i],
#                                            fuzzy_set_b.memberships[j])
#
# - Double loop: for each pair (element from A, element from B)
# - Take minimum of their memberships
#
# EXAMPLE:
# A = {(1, 0.2), (2, 0.7), (3, 1.0)}
# B = {(1, 0.5), (2, 0.4), (3, 0.9)}
#
# Cartesian Product A × B:
#        B:  1     2     3
# A: 1  [0.2,  0.2,  0.2]  ← min(0.2, each B membership)
#    2  [0.5,  0.4,  0.7]  ← min(0.7, each B membership)
#    3  [0.5,  0.4,  0.9]  ← min(1.0, each B membership)
#
# R(2,3) = min(0.7, 0.9) = 0.7
# R(1,1) = min(0.2, 0.5) = 0.2
#
# INTERPRETATION: Relationship strength limited by weaker membership
#
#         return FuzzyRelation(relation_matrix,
#                            fuzzy_set_a.elements,
#                            fuzzy_set_b.elements)
#
# ============================================================================
# MAX-MIN COMPOSITION
# ============================================================================
#
#     @staticmethod
#     def max_min_composition(r1, r2):
#         """
#         Max-min composition of two fuzzy relations
#         R(x,z) = max_y[min(R1(x,y), R2(y,z))]
#         R1: relation between X and Y
#         R2: relation between Y and Z
#         """
#
# - Chains two relations: X→Y and Y→Z to get X→Z
# - For each (x,z) pair, considers all intermediate y values
# - Takes MIN for each path, then MAX across all paths
#
# ANALOGY:
# - R1: "Students to Courses" (how interested each student is in courses)
# - R2: "Courses to Jobs" (how relevant each course is to jobs)
# - Composition: "Students to Jobs" (how suitable each student is for jobs)
#
#         if r1.y_elements != r2.x_elements:
#             raise ValueError("Relations cannot be composed: Y dimensions don't match")
#
# - Relations must share middle set (Y)
# - R1 output (Y) must match R2 input (Y)
#
#         x_size = len(r1.x_elements)
#         z_size = len(r2.y_elements)
#         y_size = len(r1.y_elements)
#
# - x_size: Number of elements in X
# - z_size: Number of elements in Z
# - y_size: Number of intermediate elements in Y
#
#         result_matrix = np.zeros((x_size, z_size))
#
# - Result is X×Z matrix
#
#         for i in range(x_size):
#             for j in range(z_size):
#
# - For each (x,z) pair in result
#
#                 max_val = 0
#                 for k in range(y_size):
#
# - Consider all intermediate y values
#
#                     min_val = min(r1.matrix[i][k], r2.matrix[k][j])
#
# - For path x→y→z: take minimum along the path
# - Bottleneck principle: chain is only as strong as weakest link
#
#                     max_val = max(max_val, min_val)
#
# - Across all paths through different y values: take maximum
# - Find the strongest path from x to z
#
#                 result_matrix[i][j] = max_val
#
# EXAMPLE CALCULATION:
# R1 (X→Y):        R2 (Y→Z):
#    Y: a  b  c       Z: x   y
# X:1 [0.3 0.8 0.5]  Y:a [0.7 0.4]
#   2 [0.6 0.2 0.7]    b [0.5 0.8]
#   3 [0.4 0.9 0.3]    c [0.6 0.3]
#
# Calculate R(1,x) (element 1 to x):
# Path through a: min(R1(1,a), R2(a,x)) = min(0.3, 0.7) = 0.3
# Path through b: min(R1(1,b), R2(b,x)) = min(0.8, 0.5) = 0.5
# Path through c: min(R1(1,c), R2(c,x)) = min(0.5, 0.6) = 0.5
# R(1,x) = max(0.3, 0.5, 0.5) = 0.5
#
# Calculate R(1,y):
# Path through a: min(0.3, 0.4) = 0.3
# Path through b: min(0.8, 0.8) = 0.8
# Path through c: min(0.5, 0.3) = 0.3
# R(1,y) = max(0.3, 0.8, 0.3) = 0.8
#
# INTERPRETATION:
# - Element 1 relates to x with strength 0.5 (best path through b or c)
# - Element 1 relates to y with strength 0.8 (best path through b)
#
#         return FuzzyRelation(result_matrix, r1.x_elements, r2.y_elements)
#
# - Returns composed relation X→Z
#
# ============================================================================
# HELPER FUNCTION
# ============================================================================
#
# def print_section(title):
#     """Helper function to print section headers"""
#     print("\n" + "="*60)
#     print(title)
#     print("="*60)
#
# - Prints formatted section headers
# - Improves output readability
#
# ============================================================================
# MAIN DEMONSTRATION
# ============================================================================
#
# def main():
#     """Main function demonstrating fuzzy set operations and relations"""
#
#     print_section("FUZZY SET OPERATIONS")
#
#     # Define fuzzy sets A and B
#     A = FuzzySet([1, 2, 3], [0.2, 0.7, 1.0])
#     B = FuzzySet([1, 2, 3], [0.5, 0.4, 0.9])
#
# - Creates two fuzzy sets on same universe {1, 2, 3}
# - Different membership values for each element
#
#     print(f"\nFuzzy Set A: {A}")
#     print(f"Fuzzy Set B: {B}")
#
# OUTPUT:
# Fuzzy Set A: { (1, 0.2), (2, 0.7), (3, 1.0) }
# Fuzzy Set B: { (1, 0.5), (2, 0.4), (3, 0.9) }
#
#     # Union
#     union_AB = A.union(B)
#     print(f"\nUnion (A ∪ B): {union_AB}")
#
# - Computes A ∪ B using max operation
# OUTPUT: Union (A ∪ B): { (1, 0.5), (2, 0.7), (3, 1.0) }
#
#     # Intersection
#     intersection_AB = A.intersection(B)
#     print(f"Intersection (A ∩ B): {intersection_AB}")
#
# - Computes A ∩ B using min operation
# OUTPUT: Intersection (A ∩ B): { (1, 0.2), (2, 0.4), (3, 0.9) }
#
#     # Complement
#     complement_A = A.complement()
#     print(f"Complement of A (A'): {complement_A}")
#
# - Computes A' using 1-x operation
# OUTPUT: Complement of A (A'): { (1, 0.8), (2, 0.3), (3, 0.0) }
#
#     print_section("FUZZY RELATIONS - CARTESIAN PRODUCT")
#
#     # Create fuzzy relation using Cartesian product
#     R = FuzzyRelation.cartesian_product(A, B)
#     print(f"\nFuzzy Relation R (A × B):")
#     print(R.matrix)
#
# - Creates relation matrix from sets A and B
# OUTPUT:
# [[0.2 0.2 0.2]
#  [0.5 0.4 0.7]
#  [0.5 0.4 0.9]]
#
#     print(f"\nRows represent elements of A: {A.elements}")
#     print(f"Columns represent elements of B: {B.elements}")
#
# - Clarifies matrix structure
#
#     print_section("MAX-MIN COMPOSITION")
#
#     # Example: Create two relations for composition
#     # R1: relation between X={1,2,3} and Y={a,b,c}
#     R1_matrix = [
#         [0.3, 0.8, 0.5],
#         [0.6, 0.2, 0.7],
#         [0.4, 0.9, 0.3]
#     ]
#     R1 = FuzzyRelation(R1_matrix, [1, 2, 3], ['a', 'b', 'c'])
#
# - R1 represents: "How element i relates to letter j"
#
#     # R2: relation between Y={a,b,c} and Z={x,y}
#     R2_matrix = [
#         [0.7, 0.4],
#         [0.5, 0.8],
#         [0.6, 0.3]
#     ]
#     R2 = FuzzyRelation(R2_matrix, ['a', 'b', 'c'], ['x', 'y'])
#
# - R2 represents: "How letter j relates to symbol k"
#
#     print("\nRelation R1 (X × Y):")
#     print(R1.matrix)
#     print(f"X = {R1.x_elements}, Y = {R1.y_elements}")
#
#     print("\nRelation R2 (Y × Z):")
#     print(R2.matrix)
#     print(f"Y = {R2.x_elements}, Z = {R2.y_elements}")
#
# - Displays both relations before composition
#
#     # Compute max-min composition
#     R_composed = FuzzyRelation.max_min_composition(R1, R2)
#     print("\nMax-Min Composition R = R1 ∘ R2 (X × Z):")
#     print(R_composed.matrix)
#     print(f"X = {R_composed.x_elements}, Z = {R_composed.y_elements}")
#
# - Computes indirect relation: X→Z through intermediate Y
# - Result shows how elements of X relate to elements of Z
#
#     # Show detailed calculation for one element
#     print("\n" + "-"*60)
#     print("Example calculation for R(1, x):")
#     print("R(1, x) = max{ min(R1(1,a), R2(a,x)), min(R1(1,b), R2(b,x)), min(R1(1,c), R2(c,x)) }")
#     print(f"        = max{{ min(0.3, 0.7), min(0.8, 0.5), min(0.5, 0.6) }}")
#     print(f"        = max{{ 0.3, 0.5, 0.5 }}")
#     print(f"        = 0.5")
#
# - Shows step-by-step calculation for one result cell
# - Demonstrates max-min composition formula
#
# ============================================================================
# APPLICATIONS OF FUZZY SETS & RELATIONS
# ============================================================================
#
# 1. DECISION MAKING:
#    - Multi-criteria evaluation with vague preferences
#    - Example: Choosing best candidate based on fuzzy ratings
#
# 2. PATTERN RECOGNITION:
#    - Matching patterns with uncertain features
#    - Example: Facial recognition with varying lighting
#
# 3. DATABASE QUERIES:
#    - Flexible queries with "approximately" conditions
#    - Example: Find "tall" employees (fuzzy height criteria)
#
# 4. CONTROL SYSTEMS:
#    - As seen in robotic arm control
#    - Handles imprecise sensor readings
#
# 5. MEDICAL DIAGNOSIS:
#    - Symptoms with varying degrees of presence
#    - Disease relationships through symptom chains
#
# 6. RECOMMENDATION SYSTEMS:
#    - User preferences (fuzzy ratings)
#    - Chained relations: users→items→categories
#
# ============================================================================
# KEY TAKEAWAYS
# ============================================================================
#
# FUZZY SETS:
# - Handle uncertainty and partial membership
# - Operations use max (union), min (intersection), 1-x (complement)
# - More flexible than crisp sets
#
# FUZZY RELATIONS:
# - Represent vague relationships between sets
# - Matrix form with degrees of relationship
# - Can be composed to find indirect relationships
#
# MAX-MIN COMPOSITION:
# - Chains relations: X→Y→Z becomes X→Z
# - Uses min for path strength, max for best path
# - Essential for reasoning with fuzzy knowledge
#
# ADVANTAGES:
# - Models human-like reasoning
# - Handles imprecise information naturally
# - Widely applicable across domains
#
# ============================================================================
# END OF EXPLANATION
# ============================================================================


FUZZY SET OPERATIONS

Fuzzy Set A: { (1, 0.2), (2, 0.7), (3, 1.0) }
Fuzzy Set B: { (1, 0.5), (2, 0.4), (3, 0.9) }

Union (A ∪ B): { (1, 0.5), (2, 0.7), (3, 1.0) }
Intersection (A ∩ B): { (1, 0.2), (2, 0.4), (3, 0.9) }
Complement of A (A'): { (1, 0.8), (2, 0.3), (3, 0.0) }

FUZZY RELATIONS - CARTESIAN PRODUCT

Fuzzy Relation R (A × B):
[[0.2 0.2 0.2]
 [0.5 0.4 0.7]
 [0.5 0.4 0.9]]

Rows represent elements of A: [1, 2, 3]
Columns represent elements of B: [1, 2, 3]

MAX-MIN COMPOSITION

Relation R1 (X × Y):
[[0.3 0.8 0.5]
 [0.6 0.2 0.7]
 [0.4 0.9 0.3]]
X = [1, 2, 3], Y = ['a', 'b', 'c']

Relation R2 (Y × Z):
[[0.7 0.4]
 [0.5 0.8]
 [0.6 0.3]]
Y = ['a', 'b', 'c'], Z = ['x', 'y']

Max-Min Composition R = R1 ∘ R2 (X × Z):
[[0.5 0.8]
 [0.6 0.4]
 [0.5 0.8]]
X = [1, 2, 3], Z = ['x', 'y']

------------------------------------------------------------
Example calculation for R(1, x):
R(1, x) = max{ min(R1(1,a), R2(a,x)), min(R1(1,b), R2(b,x)), min(R1(1,c), R2(c,x)) }
        = max{ min(0.3, 0.7), 